In [1]:
!pip list

Package                       Version
----------------------------- --------
appnope                       0.1.3
asttokens                     2.2.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
brotlipy                      0.7.0
certifi                       2023.5.7
cffi                          1.15.1
charset-normalizer            2.0.4
cryptography                  39.0.1
debugpy                       1.5.1
decorator                     5.1.1
entrypoints                   0.4
executing                     1.2.0
filelock                      3.9.0
gmpy2                         2.1.2
idna                          3.4
ipykernel                     6.15.0
ipython                       8.12.0
jedi                          0.18.2
Jinja2                        3.1.2
jupyter-client                7.3.4
jupyter_core                  5.3.0
MarkupSafe                    2.1.1
matplotlib-inline             0.1.6
mpmath                        1.2.1
nest-asyncio       

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

In [5]:
"""
Word Processing
"""

word_list = list(set(" ".join(sentences).split()))
word_dict = {w:i for i, w in enumerate(word_list)}
number_dict = {i:w for i, w in enumerate(word_list)}
n_class = len(word_dict)
print(f'word_list : {word_list}')
print(f'word_dict : {word_dict}')
print(f'number_dict : {number_dict}')
print(f'n_class : {n_class}')

word_list : ['cat', 'i', 'milk', 'love', 'dog', 'coffee', 'hate', 'like', 'you']
word_dict : {'cat': 0, 'i': 1, 'milk': 2, 'love': 3, 'dog': 4, 'coffee': 5, 'hate': 6, 'like': 7, 'you': 8}
number_dict : {0: 'cat', 1: 'i', 2: 'milk', 3: 'love', 4: 'dog', 5: 'coffee', 6: 'hate', 7: 'like', 8: 'you'}
n_class : 9


In [6]:
"""
TextRNN Parameter
"""
batch_size = len(sentences)
n_step = 2 #학습하려고 하는 문장의 길이 -1
n_hidden = 5 #은닉층 사이즈

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

In [7]:
input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)

/var/folders/lc/j0l_nf3s0pb9j49j6xsdn_p00000gn/T/ipykernel_11862/178811079.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343685278/work/torch/csrc/utils/tensor_new.cpp:248.)
  input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)


In [9]:
"""
TextRNN
"""

class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
        self.Softmax = nn.Softmax(dim=1)
    
    def forward(self, hidden, x):
        x = x.transpose(0, 1)
        outputs, hidden = self.rnn(x, hidden)
        outputs = outputs[-1] #최종 예측 hidden layer
        model = torch.mm(outputs, self.W) + self.b #최종 예측의 최종 출력 layer
        return model

In [10]:
"""
Training
"""
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
    output = model(hidden, input_batch)
    loss = criterion(output, target_batch)

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0100 cost = 1.034207
Epoch: 0200 cost = 0.204521
Epoch: 0300 cost = 0.048151
Epoch: 0400 cost = 0.025182
Epoch: 0500 cost = 0.016154


/Users/choejaehun/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [11]:
input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']
